In [ ]:

import numpy as onp
onp.set_printoptions(precision=3,suppress=True)
from sklearn.metrics import mean_squared_error

import jax
import jax.numpy as np
from jax import grad, jit, vmap, device_put, random
from flax import linen as nn


import matplotlib.pyplot as plt
import matplotlib as mpl
# https://matplotlib.org/3.1.1/gallery/style_sheets/style_sheets_reference.html
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 25
mpl.rcParams['font.family'] = 'Times New Roman'
cmap = plt.cm.get_cmap('bwr')


from tabulate import tabulate

import sys
sys.path.append('../kernel')
from jaxkern import normalize_K

from plt_utils import plt_savefig, plt_scaled_colobar_ax
from gpax import *

In [ ]:
## Parameters

M = 1
n_train = 50
n_test = 100
ylim = (-.5, 2.5)
xlim = (-1.2, 1.2)
σn = .003
ℓ = .2
bsz = 20


## Data

onp.random.seed(0)
key = jax.random.PRNGKey(0)

def f(X,slope=2, intercept=0): return .3*np.sin(5*X) + slope*X + intercept
X = np.sort(random.uniform(key, (n_train, 1)), axis=0)*2-1
y = f(X,2) + random.normal(key, (n_train, M))*σn
data = (X, y)
print(X.shape, y.shape)

Xs = np.linspace(xlim[0], xlim[1], n_test).reshape(-1,1)

for i in range(M):
    plt.scatter(X, y[:,i])

In [ ]:
class ExMTGP(GPR):
    def setup(self):
        self.k = CovICM(kt_kwargs={'output_dim': M})
        self.mean_fn = MeanConstant(M, init_val_m=0.)
        self.lik = LikMultipleNormalKron(M)
        
class SvMTGP(SVGP):

    def setup(self):
        self.d = self.Xu_initial.shape[1]
        self.n_inducing = self.Xu_initial.shape[0]
        self.mean_fn = MeanConstant(M, init_val_m=1.)
        self.k = CovICM(kt_kwargs={'output_dim': M})
        self.lik = LikNormal()
        self.Xu = self.param('Xu', lambda k, s: self.Xu_initial,
                             (self.n_inducing, self.d))
        self.q = VariationalMultivariateNormal(self.n_inducing*M)
    

# n = 10
# key = random.PRNGKey(0)
# X = np.sort(random.uniform(key, (10, 1)), axis=0)
# y = random.normal(key, X.shape)*.2
# y = y*np.arange(M).reshape(1,-1)
# data = (X, y)

# model = ExMTGP(data)
# params = model.get_init_params(key)
# model = model.bind(params)
# model.pred_f(X[:2])


model = SvMTGP(n_data=len(X), Xu_initial=X[:10])
params = model.get_init_params(key, n_tasks=M)
model = model.bind(params)
model.pred_f(X[:2]), model.mll((X[:2], y[:2]))

In [ ]:
colors_b = [cmap(.1), cmap(.3)]
colors_r = [cmap(.9), cmap(.7)]
fig, axs = plt.subplots(4,2,figsize=(20,20))


for it, intercept in enumerate([.1,.5,1,2]):
    
    def f_gen(X): return f(X,slope=0, intercept=intercept)
    
    y = f_gen(X) + random.normal(key, (n_train, M))*σn
    data = (X, y)
    
    n_batches, batches = get_data_stream(
        key, bsz, X, y)

    ## Training
    model = SvMTGP(n_data=len(X),
                   Xu_initial=X[random.randint(key, (20,), 0, n_train)])
    params = model.get_init_params(key)
    

    @jax.jit
    def train_step(step, opt, batch):
        def f(params):
            return -model.apply(params, batch, method=model.mll)
        fg_fn = jax.value_and_grad(f)
        fx, grad = fg_fn(opt.target)
        opt = opt.apply_gradient(grad)
        log = {'step': step,
               'loss': fx}
        return opt, log

    num_steps = 700
    opt = flax_create_optimizer(
        params, 'Adam', {'learning_rate': .03})
    for j in range(num_steps):
        for k in range(n_batches):
            step = j*n_batches+k
            batch = next(batches)
            opt, log = train_step(step, opt, batch)
            if step%(20*n_batches)==0:
                print(f'[{log["step"]:3}]\t'
                      f'Loss={log["loss"]:.3f}\t')

    params = opt.target

    ## Plotting
    model = model.bind(params)
    mll = model.mll(data)
    μ, Σ = model.pred_f(Xs, full_cov=False)
    μ = μ.flatten()
    std = np.sqrt(Σ).flatten()

    ax = axs[it,0]
    ax.plot(Xs, μ, color=colors_b[0], lw=2)
    ax.fill_between(Xs.flatten(), μ-2*std, μ+2*std, alpha=.2, color=colors_b[0])
    ax.plot(Xs, f_gen(Xs), linewidth=1, color='k', linestyle='dashed')
    ax.scatter(X, y, marker='x', color=colors_r[0], s=50)
    ax.grid()
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    title = ' $-mll$'+f'={-mll.item():.2f}, ' + \
            '$\ell$'+f'={model.k.kx.ℓ.item():.2f}, ' + \
            '$m$'+f'={model.mean_fn.c[0]:.2f}, ' + \
            '$\sigma_n$'+f'={model.lik.σ2.item():.4f}'
    ax.set_title(title)

    ax = axs[it,1]
    XX = np.vstack((X, Xs))
    K = model.k(XX)
    im = ax.imshow(normalize_K(K), cmap=cmap)
    fig.colorbar(im, cax=plt_scaled_colobar_ax(ax))
    ax.set_title('$K(X, Xs)$')



fig.tight_layout()
plt_savefig(fig, 'summary/assets/plt_svgp_mean.png')
